# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI.

In [2]:
import wikipedia
import numpy as np
import os
import json
import requests

In [6]:
code2lang = wikipedia.languages()
langs = ['fr', 'fi', 'no', 'it' , 'ru', 'sc', 'sr', 'pl', 'myv', 'ik']
[(lang, code2lang[lang]) for lang  in langs]

[('fr', 'français'),
 ('fi', 'suomi'),
 ('no', 'norsk'),
 ('it', 'italiano'),
 ('ru', 'русский'),
 ('sc', 'sardu'),
 ('sr', 'српски / srpski'),
 ('pl', 'polski'),
 ('myv', 'эрзянь'),
 ('ik', 'Iñupiatun')]

In [7]:
def load_with_disambigution(page):
    try:
        p = wikipedia.page(page)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = wikipedia.page(random_option)
    return p

In [8]:
def get_texts_for_lang(lang, n=100): # функция для скачивания статей из википедии
    # вот так можно зафиксировать язык
    wikipedia.set_lang(lang)
    wiki_content = []
    
    # random достает только ссылки на статьи
    pages = wikipedia.random(n)
    
    for page_name in pages:
        try:
            # чтобы загрузить контент статьи можно вызвать функцию .page
            page = load_with_disambigution(page_name)
        
        except Exception as e:
            print('Skipping page {}'.format(page_name), e)
            continue

        wiki_content.append(f'{page.title}\n{page.content.replace("==", "")}')

    return wiki_content

In [9]:
wiki_texts = {}

for lang in langs:
    try:
        wiki_texts[lang] = get_texts_for_lang(lang, 20)
    except Exception as e:
        print('ERROR ON - ', lang, e)
        continue
    
    print(lang, len(wiki_texts[lang]))

/Users/ignatenkodarja/Library/Python/3.7/lib/python/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/ignatenkodarja/Library/Python/3.7/lib/python/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


fr 20
fi 20
no 20
it 20
Skipping page Рогачёвская волость "Рогачёвская волость" may refer to: 
Рогачёвская волость (Александровский уезд)
Рогачёвская волость (Новоград-Волынский уезд)
Рогачёвская волость (Воронежский уезд)
Рогачёвская волость (Дмитровский уезд)
ru 19
sc 20
sr 20
pl 20
myv 20
ik 20


In [10]:
from collections import defaultdict
from string import digits
import re

digits = set(digits)

# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуру через DependencyGraph, выберите отношение (из тех, что не упоминались на семинаре) и найдите самые частотные пары слов, связанных этим отношением. 

Для самой частотной пары вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)`) В итоге у вас должен получится список пары: слово1 и все его подзависимые - слово2 и все его подзависимые.

Визуализируйте самое длинное предложение через networkx